# ipywidgets

## Notebook to PDF Converter Without Callback

[FileLink 可以改](https://github.com/voila-dashboards/voila/issues/578)。同一個 issue，有人示範了 download button，不需要知道 file encoding。下面這個版本可以成功佈署到 Voila 上

In [2]:
from ipywidgets import FileUpload, Button, Output
from IPython.display import display, HTML
import subprocess, os, base64

output = Output()
fileInput = FileUpload()
convertButton = Button(description='Convert')

def convert(b):
    with output:
        print('Your PDF will be ready in a few seconds. ')
        nbName = list(fileInput.value.keys())[0]
        pdfName = nbName.replace('ipynb', 'pdf')
        with open(nbName, 'w+b') as f:
            f.write(fileInput.data[0])
        proc = subprocess.run(['jupyter', 'nbconvert', nbName, '--to', 'pdf'])
        if proc.returncode == 1:
            print('LaTeX compiling error!')
        else:
            with open(pdfName, "rb") as f:
                data = f.read()
                b64 = base64.b64encode(data)
                payload = b64.decode()
            display(HTML(f'''<a download="{pdfName}" href="data:text/csv;base64,{payload}" target="_blank">  
                                <button class="p-Widget jupyter-widgets jupyter-button widget-button"> Download PDF </button>
                            </a>'''))

convertButton.on_click(convert)

display(fileInput, convertButton, output) 

FileUpload(value={}, description='Upload')

Button(description='Convert', style=ButtonStyle())

Output()

## Notebook to PDF Converter

用 Javascript ```window.location.href``` 可以知道當前 server url。下面這個 app 在 JupyterLab 是完全可以跑的，不過 [Volia 一遇到 Javascript 會有很多問題](https://github.com/voila-dashboards/voila/issues/108)，所以用 Voila 佈署失敗。如果把 ```alert("aaa");``` 插在 callback 的最前面，實驗顯示 Voila 根本完全沒有執行 callback。這個 [issue](https://github.com/voila-dashboards/voila-material/issues/18) 提到 Voila 遇到 callback 會失敗的問題。目前還沒有修好

In [8]:
from ipywidgets import FileUpload, Button, Output
from IPython.display import display, FileLink, HTML, Javascript
import subprocess, os

env = 'lab'   # change to 'voila' before deploying

output = Output()
fileInput = FileUpload()
convertButton = Button(description='Convert')

def convert(b):
    with output:
        print('Your PDF will be ready in a few seconds. ')
        nbName = list(fileInput.value.keys())[0]
        pdfName = nbName.replace('ipynb', 'pdf')
        with open(nbName, 'w+b') as f:
            f.write(fileInput.data[0]) 
        proc = subprocess.run(['jupyter', 'nbconvert', nbName, '--to', 'pdf'])
        if proc.returncode == 1:
            print('LaTeX compiling error!')
        else:
            pdfPath = '/'.join(['tree'] + str(FileLink(pdfName)).split('/')[3:])
            display(HTML('''
                        <html>
                        <body>
                        <script type="text/javascript">
                            function setHref() {
                                var server = window.location.href;
                                var homeIdx = server.indexOf("%s");
                                server = server.slice(0, homeIdx);
                                var href = server + "%s";
                                open(href);
                            }
                        </script>
                        <form name="buttonForm">
                            <button class="p-Widget jupyter-widgets jupyter-button widget-button" onClick="setHref();return true;">
                                Download PDF
                            </button>
                        </form>
                        </body>
                        </html>
                         '''% (env, pdfPath)))

convertButton.on_click(convert)

display(fileInput, convertButton, output) 

FileUpload(value={}, description='Upload')

Button(description='Convert', style=ButtonStyle())

Output()

## [Download Button Example in ipywidgets](https://stackoverflow.com/questions/61708701/how-to-download-a-file-using-ipywidget-button)

This works in Voila, but to download PDF this way one needs to know the encoding of the PDF. 

In [2]:
from ipywidgets import HTML
from IPython.display import display

import base64

res = 'computed results'

#FILE
filename = 'res.txt'
b64 = base64.b64encode(res.encode())
payload = b64.decode()

#BUTTONS
html_buttons = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="{filename}" href="data:text/csv;base64,{payload}" download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
</a>
</body>
</html>
'''

html_button = html_buttons.format(payload=payload,filename=filename)
display(HTML(html_button))

HTML(value='<html>\n<head>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n</head>\n<bo…

## Get Server URL and Pass to Python

* 下面這段 code 來自[這裡](https://stackoverflow.com/questions/31818127/can-a-jupyter-ipython-notebook-take-arguments-in-the-url/37134476#37134476)。可能在 Jupyter Notebook 可以跑可是在 JupyterLab 不行因為 JavaScript 被 block 了
* 可能有辦法透過 [pyviz_comms](https://github.com/holoviz/pyviz_comms) 得到 URL（sandbox-stable 上有安裝），不過能不能在 Voila 上跑是另一個問題

In [4]:
%%javascript

IPython.notebook.kernel.execute("url = '" + window.location + "'")

<IPython.core.display.Javascript object>

## [Create New Cell Programmatically](https://discourse.jupyter.org/t/how-to-programmatically-add-serveral-new-cells-in-a-notebook-in-jupyterlab/4323)

In [3]:
def create_new_cell(contents):
    from IPython.core.getipython import get_ipython
    shell = get_ipython()
    payload = dict(
        source='set_next_input',
        text=contents,
        replace=False,
    )
    shell.payload_manager.write_payload(payload, single = False)

contents = 'print("Hello World")'
create_new_cell(contents=contents)

In [ ]:
print("Hello World")